In [ ]:
# Wine Quality API Client Examples

This notebook demonstrates how to interact with the Wine Quality Prediction API using different approaches.

## Setup

First, let's import the necessary libraries:

```python
import requests
import json
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
```

## API Base URL
```python
base_url = "http://127.0.0.1:8000"
```

## REST API Examples (FastAPI)

### Get Model Information

```python
response = requests.get(f"{base_url}/info")
model_info = response.json()
print(json.dumps(model_info, indent=2))
```

### Make a Prediction

```python
# Test data for a red wine
red_wine_data = {
    "volatile_acidity": 0.7,
    "chlorides": 0.08,
    "free_sulfur_dioxide": 15,
    "total_sulfur_dioxide": 30,
    "density": 0.9978,
    "pH": 3.2,
    "sulphates": 0.8,
    "alcohol": 10.5,
    "type_white": 0  # Red wine
}

response = requests.post(f"{base_url}/predict", json=red_wine_data)
red_wine_result = response.json()
print(f"Red Wine Quality Prediction: {red_wine_result['prediction']}")

# Test data for a white wine
white_wine_data = {
    "volatile_acidity": 0.3,
    "chlorides": 0.04,
    "free_sulfur_dioxide": 30,
    "total_sulfur_dioxide": 120,
    "density": 0.9925,
    "pH": 3.3,
    "sulphates": 0.5,
    "alcohol": 12.0,
    "type_white": 1  # White wine
}

response = requests.post(f"{base_url}/predict", json=white_wine_data)
white_wine_result = response.json()
print(f"White Wine Quality Prediction: {white_wine_result['prediction']}")
```

## GraphQL Example

### Get Model Information with GraphQL

```python
query = """
query {
  modelInfo {
    modelType
    featureSet
  }
}
"""

response = requests.post(f"{base_url}/graphql", json={"query": query})
graphql_info = response.json()
print(json.dumps(graphql_info, indent=2))
```

### Make a Prediction with GraphQL

```python
mutation = """
mutation PredictQuality($input: WineQualityInput!) {
  predictQuality(wineInput: $input) {
    prediction
    featuresUsed {
      volatileAcidity
      chlorides
      freeSulfurDioxide
      totalSulfurDioxide
      density
      pH
      sulphates
      alcohol
      typeWhite
      totalSulfurDioxideToFreeSulfurDioxide
    }
  }
}
"""

variables = {
    "input": {
        "volatileAcidity": 0.5,
        "chlorides": 0.06,
        "freeSulfurDioxide": 20,
        "totalSulfurDioxide": 80,
        "density": 0.9950,
        "pH": 3.25,
        "sulphates": 0.7,
        "alcohol": 11.0,
        "typeWhite": 1
    }
}

response = requests.post(f"{base_url}/graphql", json={"query": mutation, "variables": variables})
graphql_prediction = response.json()
print(f"GraphQL Wine Quality Prediction: {graphql_prediction['data']['predictQuality']['prediction']}")
```

## Batch Predictions Example

Let's create a small dataset of wines and get predictions for all of them:

```python
# Create a small dataset of wines
wines = [
    # Red wines
    {"volatile_acidity": 0.7, "chlorides": 0.08, "free_sulfur_dioxide": 15, "total_sulfur_dioxide": 30, "density": 0.9978, "pH": 3.2, "sulphates": 0.8, "alcohol": 9.5, "type_white": 0},
    {"volatile_acidity": 0.5, "chlorides": 0.07, "free_sulfur_dioxide": 12, "total_sulfur_dioxide": 35, "density": 0.9965, "pH": 3.4, "sulphates": 0.7, "alcohol": 10.0, "type_white": 0},
    {"volatile_acidity": 0.6, "chlorides": 0.09, "free_sulfur_dioxide": 10, "total_sulfur_dioxide": 40, "density": 0.9980, "pH": 3.3, "sulphates": 0.9, "alcohol": 11.0, "type_white": 0},
    {"volatile_acidity": 0.4, "chlorides": 0.06, "free_sulfur_dioxide": 20, "total_sulfur_dioxide": 45, "density": 0.9950, "pH": 3.1, "sulphates": 0.6, "alcohol": 12.0, "type_white": 0},
    {"volatile_acidity": 0.8, "chlorides": 0.10, "free_sulfur_dioxide": 8, "total_sulfur_dioxide": 25, "density": 0.9990, "pH": 3.5, "sulphates": 1.0, "alcohol": 9.0, "type_white": 0},
    
    # White wines
    {"volatile_acidity": 0.3, "chlorides": 0.04, "free_sulfur_dioxide": 30, "total_sulfur_dioxide": 120, "density": 0.9925, "pH": 3.3, "sulphates": 0.5, "alcohol": 12.0, "type_white": 1},
    {"volatile_acidity": 0.25, "chlorides": 0.03, "free_sulfur_dioxide": 35, "total_sulfur_dioxide": 130, "density": 0.9930, "pH": 3.2, "sulphates": 0.4, "alcohol": 11.5, "type_white": 1},
    {"volatile_acidity": 0.35, "chlorides": 0.05, "free_sulfur_dioxide": 25, "total_sulfur_dioxide": 110, "density": 0.9920, "pH": 3.4, "sulphates": 0.6, "alcohol": 12.5, "type_white": 1},
    {"volatile_acidity": 0.2, "chlorides": 0.02, "free_sulfur_dioxide": 40, "total_sulfur_dioxide": 140, "density": 0.9915, "pH": 3.1, "sulphates": 0.3, "alcohol": 13.0, "type_white": 1},
    {"volatile_acidity": 0.4, "chlorides": 0.06, "free_sulfur_dioxide": 20, "total_sulfur_dioxide": 100, "density": 0.9935, "pH": 3.5, "sulphates": 0.7, "alcohol": 11.0, "type_white": 1},
]

# Make predictions for all wines
predictions = []
for wine in wines:
    response = requests.post(f"{base_url}/predict", json=wine)
    result = response.json()
    predictions.append({
        "type": "Red" if wine["type_white"] == 0 else "White",
        "alcohol": wine["alcohol"],
        "prediction": result["prediction"]
    })

# Convert to DataFrame
df = pd.DataFrame(predictions)
print(df)

# Plot results
plt.figure(figsize=(10, 6))
sns.barplot(x="type", y="prediction", data=df, palette=["darkred", "lightyellow"])
plt.title("Wine Quality Predictions by Type")
plt.xlabel("Wine Type")
plt.ylabel("Predicted Quality")
plt.show()

plt.figure(figsize=(10, 6))
sns.scatterplot(x="alcohol", y="prediction", hue="type", data=df, palette=["darkred", "lightyellow"])
plt.title("Wine Quality Predictions by Alcohol Content")
plt.xlabel("Alcohol Content (%)")
plt.ylabel("Predicted Quality")
plt.show()
```

## Using the Batch Prediction API

A more efficient way to get predictions for multiple wines is to use the batch prediction endpoint:

```python
# Create a DataFrame with wine data
wine_df = pd.DataFrame([
    # Red wines
    {"volatile_acidity": 0.7, "chlorides": 0.08, "free_sulfur_dioxide": 15, "total_sulfur_dioxide": 30, "density": 0.9978, "pH": 3.2, "sulphates": 0.8, "alcohol": 9.5, "type_white": 0},
    {"volatile_acidity": 0.5, "chlorides": 0.07, "free_sulfur_dioxide": 12, "total_sulfur_dioxide": 35, "density": 0.9965, "pH": 3.4, "sulphates": 0.7, "alcohol": 10.0, "type_white": 0},
    {"volatile_acidity": 0.6, "chlorides": 0.09, "free_sulfur_dioxide": 10, "total_sulfur_dioxide": 40, "density": 0.9980, "pH": 3.3, "sulphates": 0.9, "alcohol": 11.0, "type_white": 0},
    
    # White wines
    {"volatile_acidity": 0.3, "chlorides": 0.04, "free_sulfur_dioxide": 30, "total_sulfur_dioxide": 120, "density": 0.9925, "pH": 3.3, "sulphates": 0.5, "alcohol": 12.0, "type_white": 1},
    {"volatile_acidity": 0.25, "chlorides": 0.03, "free_sulfur_dioxide": 35, "total_sulfur_dioxide": 130, "density": 0.9930, "pH": 3.2, "sulphates": 0.4, "alcohol": 11.5, "type_white": 1},
])

# Save to a temporary CSV file
temp_csv_path = "temp_batch_wines.csv"
wine_df.to_csv(temp_csv_path, index=False)

# Option 1: Get batch predictions as JSON
with open(temp_csv_path, 'rb') as f:
    response = requests.post(
        f"{base_url}/batch-predict", 
        files={'file': ('wines.csv', f, 'text/csv')}
    )

batch_results = response.json()
print("Batch Prediction Results:")
print(f"Number of predictions: {len(batch_results['predictions'])}")
print(f"Success rate: {batch_results['success_rate'] * 100}%")
print(f"Predictions: {batch_results['predictions']}")

# Option 2: Get batch predictions as CSV with predictions included
with open(temp_csv_path, 'rb') as f:
    response = requests.post(
        f"{base_url}/batch-predict", 
        files={'file': ('wines.csv', f, 'text/csv')},
        params={'download': 'true'}
    )

# Save the CSV with predictions
result_csv_path = "wines_with_predictions.csv"
with open(result_csv_path, 'wb') as f:
    f.write(response.content)

# Read the predictions CSV and display
result_df = pd.read_csv(result_csv_path)
print("\nPredictions CSV Result:")
print(result_df)

# Clean up temporary files
import os
os.remove(temp_csv_path)
os.remove(result_csv_path)

# Visualize the batch predictions
result_df['Wine Type'] = result_df['type_white'].map({0: 'Red', 1: 'White'})
plt.figure(figsize=(10, 6))
sns.barplot(x='Wine Type', y='prediction', data=result_df, palette=["darkred", "lightyellow"])
plt.title("Batch Wine Quality Predictions by Type")
plt.ylabel("Predicted Quality")
plt.tight_layout()
plt.show()
```

## Feature Importance Analysis

We can investigate which features most affect the predictions:

```python
# Create a baseline wine
baseline_wine = {
    "volatile_acidity": 0.5,
    "chlorides": 0.06,
    "free_sulfur_dioxide": 20,
    "total_sulfur_dioxide": 80,
    "density": 0.9950,
    "pH": 3.25,
    "sulphates": 0.7,
    "alcohol": 11.0,
    "type_white": 1
}

# Make baseline prediction
response = requests.post(f"{base_url}/predict", json=baseline_wine)
baseline_result = response.json()
baseline_prediction = baseline_result["prediction"]
print(f"Baseline Quality Prediction: {baseline_prediction}")

# Test each feature impact
feature_impacts = []
for feature in baseline_wine:
    # Skip categorical feature
    if feature == "type_white":
        continue
        
    # Create test wine with increased feature value (+20%)
    test_wine = baseline_wine.copy()
    test_wine[feature] = test_wine[feature] * 1.2
    
    # Make prediction
    response = requests.post(f"{base_url}/predict", json=test_wine)
    test_result = response.json()
    test_prediction = test_result["prediction"]
    
    # Calculate impact
    impact = test_prediction - baseline_prediction
    feature_impacts.append({
        "feature": feature,
        "baseline": baseline_wine[feature],
        "modified": test_wine[feature],
        "impact": impact
    })

# Convert to DataFrame
impact_df = pd.DataFrame(feature_impacts)
print(impact_df)

# Plot feature impacts
plt.figure(figsize=(12, 6))
sns.barplot(x="feature", y="impact", data=impact_df)
plt.title("Feature Impact on Wine Quality Prediction")
plt.xlabel("Feature (20% Increase)")
plt.ylabel("Change in Predicted Quality")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
```